In [ ]:
!pip install langchain langchain-google-genai langgraph python-dotenv
!pip install google-generativeai

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "Your API Key"

In [ ]:
from langchain.tools import BaseTool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.messages import HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent
import os

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0.7,
    google_api_key=os.getenv("GEMINI_API_KEY")
)

class SimpleKnowledgeBaseTool(BaseTool):
    name: str = "simple_knowledge_base"
    description: str = "Retrieves basic information about AI concepts."

    def _run(self, query: str):
        knowledge = {
            "MCP": "Model Context Protocol (MCP) is an open standard by Anthropic designed to connect AI assistants with external data sources, enabling real-time, context-rich interactions.",
            "RAG": "Retrieval-Augmented Generation (RAG) enhances LLM responses by dynamically retrieving relevant external documents."
        }
        return knowledge.get(query, "I don't have information on that topic.")

    async def _arun(self, query: str):
        return self._run(query)

kb_tool = SimpleKnowledgeBaseTool()
tools = [kb_tool]
graph = create_react_agent(model, tools)

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def chat_with_agent():
    inputs = {"messages": []}

    print("🤖 MCP-Like Assistant ready! Type 'exit' to quit.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("👋 Ending chat.")
            break

        from langchain.schema.messages import HumanMessage, AIMessage
        inputs["messages"].append(HumanMessage(content=user_input))

        async for state in graph.astream(inputs, stream_mode="values"):
            last_message = state["messages"][-1]
            if isinstance(last_message, AIMessage):
                print("\nAgent:", last_message.content)

        inputs["messages"] = state["messages"]

await chat_with_agent()

🤖 MCP-Like Assistant ready! Type 'exit' to quit.

You: What is MCP?

Agent: 

Agent: Model Context Protocol (MCP) is an open standard by Anthropic designed to connect AI assistants with external data sources, enabling real-time, context-rich interactions.

You: exit
👋 Ending chat.
